In [4]:
import numpy as np
import pickle
import time 
import timeit
import shutil 
import h5py
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
from functools import partial
import glob
import os
import pandas as pd
import statistics as sta
from scipy.stats import circstd, circmean
from scipy.signal import savgol_filter
from matplotlib.animation import FuncAnimation
#import changefinder
import ruptures as rpt
import seaborn as sns

from matplotlib import animation

from pathlib import Path


In [13]:
loc = "~/vmodel_output/"
file = "noPred_noMultCol_v2pol_astr_dphi.csv"
method="bs"
plt.style.use('default')
pol = pd.read_csv(loc+file, header=None, index_col=False)
pol2 = pd.read_csv(loc+file, header=None, index_col=False)
relax_T3=[]

In [17]:
pol

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.008192,0.009313,0.005533,0.005676,0.008562,0.009315,0.005006,0.004736,0.004691,0.004920,0.003321,0.004821,0.000922,0.002654,0.002584,0.002836,0.003245,0.001124,0.001925,0.001456
1,0.011107,0.006319,0.005904,0.006596,0.006449,0.007703,0.006366,0.002346,0.004627,0.005633,0.002603,0.004005,0.003582,0.004022,0.001981,0.001457,0.002987,0.003162,0.001911,0.003278
2,0.014796,0.015018,0.009270,0.008449,0.005809,0.008908,0.006238,0.007356,0.004194,0.005669,0.004308,0.004409,0.004534,0.003198,0.003272,0.004097,0.003092,0.003024,0.004295,0.005866
3,0.016064,0.022269,0.020460,0.010699,0.010321,0.009318,0.011273,0.010033,0.008323,0.005847,0.009187,0.005743,0.007468,0.008072,0.009128,0.003491,0.004636,0.003786,0.003253,0.003781
4,0.047002,0.025218,0.023854,0.021123,0.013630,0.014027,0.014302,0.014126,0.011297,0.012375,0.008257,0.005086,0.005539,0.008817,0.003939,0.005070,0.004488,0.005565,0.006010,0.004885
5,0.066364,0.043490,0.037302,0.038252,0.038252,0.018154,0.021752,0.013505,0.012961,0.011247,0.010416,0.012406,0.008687,0.009189,0.010092,0.009438,0.009470,0.010484,0.007698,0.005789
6,0.135926,0.085705,0.074303,0.036357,0.035317,0.034455,0.020095,0.018623,0.021739,0.018747,0.012498,0.012159,0.014349,0.008577,0.013259,0.009794,0.008687,0.004969,0.007941,0.009227
7,0.235889,0.191910,0.077714,0.076183,0.047537,0.035076,0.035597,0.029297,0.019992,0.027246,0.022084,0.015941,0.015941,0.014754,0.010645,0.010220,0.007439,0.010676,0.005898,0.006960
8,0.338728,0.220023,0.203472,0.131719,0.122270,0.069646,0.054313,0.032578,0.022725,0.023858,0.019253,0.014533,0.020287,0.016113,0.019468,0.015360,0.014019,0.011621,0.014593,0.010974
9,0.462887,0.346057,0.300571,0.178915,0.119552,0.099790,0.062568,0.049518,0.040287,0.043920,0.025301,0.026687,0.028069,0.021199,0.017236,0.016946,0.013700,0.014237,0.010867,0.009454


In [14]:
## ALL THE CODE BELOW - COMPUTES THE RELAXATION TIME ###
mean_pol0=np.transpose(pol2).mean(axis=1)
print(mean_pol0)
mean_pol1=[np.mean(mean_pol0)]*time
flat_list = [item for sublist in pol2 for item in sublist]
std_pol1=np.std(flat_list)
    
for k in range(nruns):
           # t_end_fnt=[index for index,value in enumerate(FNT[k]) if value == 1][-1]
    points=pol[k]         
    if method=="dp":
                #Changepoint detection with dynamic programming search method
        model = "l1"  
        algo = rpt.Dynp(model=model, min_size=3, jump=5).fit(points)
    elif method=="bs":
                #Changepoint detection with the Binary Segmentation search method
        model = "l2"  
        algo = rpt.Binseg(model=model).fit(points)
    my_bkps = algo.predict(n_bkps=10)
          #  relax_t[k] = my_bkps[-2]-t_end_fnt  #### <----- WE DO NOT DO IT SINCE IT CAN INCREASE THE VARIATION IN THE VALUES, WE WANT TO KEEP IT CLEAN
           
            #############################################################################
            #############################################################################
           #### Lower Control Limits
    yhat2 = savgol_filter(mean_pol1-3*std_pol1-points, 333, 2) #smoothes 
    l2=np.where(yhat2 <0)[0]
    if len(np.array(l2))>0:
        i=1
        while (l2[i]-l2[i-1]==1) and (i<len(np.array(l2))-1):
                      i=i+1
    else:
        i=0

    if i==len(np.array(l2))-1 or i==0:
        relax3_t[k]=np.nan # time
    else:
        relax3_t[k]=l2[i]


relax_T3.append(relax3_t)

0     0.403586
1     0.377528
2     0.348663
3     0.306914
4     0.292899
5     0.250889
6     0.220051
7     0.187456
8     0.161191
9     0.134493
10    0.104048
11    0.097935
12    0.074731
13    0.059606
14    0.047335
15    0.036738
16    0.034297
17    0.028151
18    0.025524
19    0.022384
dtype: float64


TypeError: can't multiply sequence by non-int of type 'module'

In [ ]:
method="bs"
plt.style.use('default')

#folders = ['/FrontA-40/', '/BackA-40/', '/SideA-40/']
folders = ['/FRONT/', '/BACK/', '/SIDE/']
cwd = os.getcwd()
#### UPLOADING DATA WITHOUT THE PREDATPR #################
#nopred_files=glob.glob('/extra2/pbartash/NoPred/*.h5')
nopred_files=glob.glob('/home/bartashevich/Documents/NoPred/*.h5')

relax_T=[]
relax_T3=[] #### YOU WILL NEED ONLY THIS #################
NND=[]
allMD=[]

In [ ]:
for atk_id in range(0,len(folders)):

    #files = glob.glob(cwd+ folders[atk_id] + '/*Dphi0.2.h5')
    files = glob.glob(cwd+ '/Exp5' + folders[atk_id] + '*Dphi0.2.h5')
    
    sorted_files=np.sort(files)
    sfiles=sorted_files #[dphi::11]

    psorted_files=np.sort(nopred_files)
    psfiles=psorted_files[2]
    m2 = h5py.File(psfiles, "r") # FILE WITHOUT A PREDATOR - ONLY ONE, SINCE THERE ARE NO ATTACKS FROM ANY DIRECTIONS

    for id in range(len(sfiles)):
        m = h5py.File(sfiles[id], "r") # FILE WITH THE PREDATOR, ITERATED OVER FRONT, BACK AND SIDE ATTACKS IN THE FOR-LOOP WITH "ID"
       
        for key in m.keys():
            print(key)

        for key2 in m2.keys():
            print(key2)

        dyn_runs=m[key]["swarm"] # WITH A PREDATOR
        dyn_runs2=m2[key2]["swarm"] # WITHOUT A PREDATOR

        nruns=dyn_runs.shape[0]
        tt=  dyn_runs2.shape[1] #dyn_runs.shape[1]

        print(dyn_runs.shape[0])
        print(dyn_runs.shape[1])
        print(dyn_runs2.shape[0])
        print(dyn_runs2.shape[1])

       ## analysis of foun data: time of fountain effect
    
        time = tt

        MD=[[]]*nruns
        FNT=[[]]*nruns
        fnt_start=[[]]*nruns
        fnt_end=[[]]*nruns
        nnd=[[]]*nruns
        iid=[[]]*nruns
        pol=[[]]*nruns
        pol2=[[]]*nruns
        elong=[[]]*nruns
        relax_t = [[]]*nruns
        relax2_t = [[]]*nruns
        relax3_t = [[]]*nruns

        for fn in range(nruns):
            for i in range(time):
                nnd[fn]=np.append(nnd[fn],dyn_runs[fn][i][13])
                iid[fn]=np.append(iid[fn],dyn_runs[fn][i][14])
                pol[fn]=np.append(pol[fn],dyn_runs[fn][i][1]) # <----- WHAT YOU NEED: POLARISATION WITH THE PREDATOR
              # pol2[fn]=np.append(pol2[fn],dyn_runs2[fn][i][1])
                elong[fn]=np.append(elong[fn],dyn_runs[fn][i][9])
        
        
        for fn in range(nruns):
            for i in range(time):
                pol2[fn]=np.append(pol2[fn],dyn_runs2[fn][i][1]) # <----- WHAT YOU NEED: POLARISATION WITHOUT THE PREDATOR

        
        ################################################################################
        ################## THE IMPORTANT STUFF STARTS BELOW ############################
        ################################################################################

        ## ALL THE CODE BELOW - COMPUTES THE RELAXATION TIME ###
        mean_pol0=np.transpose(pol2).mean(axis=1)
        mean_pol1=[np.mean(mean_pol0)]*time
        flat_list = [item for sublist in pol2 for item in sublist]
        std_pol1=np.std(flat_list)
    
        for k in range(nruns):
           # t_end_fnt=[index for index,value in enumerate(FNT[k]) if value == 1][-1]
            points=pol[k]         
            if method=="dp":
                #Changepoint detection with dynamic programming search method
                model = "l1"  
                algo = rpt.Dynp(model=model, min_size=3, jump=5).fit(points)
            elif method=="bs":
                #Changepoint detection with the Binary Segmentation search method
                model = "l2"  
                algo = rpt.Binseg(model=model).fit(points)
            my_bkps = algo.predict(n_bkps=10)
          #  relax_t[k] = my_bkps[-2]-t_end_fnt  #### <----- WE DO NOT DO IT SINCE IT CAN INCREASE THE VARIATION IN THE VALUES, WE WANT TO KEEP IT CLEAN
           
            #############################################################################
            #############################################################################
           #### Lower Control Limits
            yhat2 = savgol_filter(mean_pol1-3*std_pol1-points, 333, 2) #smoothes 
            l2=np.where(yhat2 <0)[0]
            if len(np.array(l2))>0:
                i=1
                while (l2[i]-l2[i-1]==1) and (i<len(np.array(l2))-1):
                      i=i+1
            else:
                i=0

            if i==len(np.array(l2))-1 or i==0:
                relax3_t[k]=np.nan # time
            else:
                relax3_t[k]=l2[i]


        relax_T3.append(relax3_t)

